In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy import integrate

In [3]:
#Keep fixed for this notebook!
m = 4  #number of alternative = number of vertices
ssb_utility = np.triu(np.ones((m, m))) - np.tril(np.ones((m, m))) # transitive preferences

In [16]:
def compute_bounds(parts: int):
    num_points = int(parts*(parts+1)*(parts+2)/6) # Only for m = 4
    points = np.empty((num_points, m))
    factors = np.empty(num_points)

    counter = 0
    for x in range(parts):
        for y in range(parts-x):
            for z in range(parts-x-y):
                points[counter] = [x, y, z, parts-x-y-z]
                if x+y+z == parts-1:
                    factors[counter] = 1
                elif x+y+z == parts-2:
                    factors[counter] = 5
                else:
                    factors[counter] = 6
                counter += 1

    points /= parts
    normals = points @ ssb_utility
    del points

    dist = normals[:, :, None]/(normals[:, :, None] - normals[:, None, :]) #distance
    above_b = 1 - dist[:, 0, 1]*dist[:, 0, 2]*dist[:, 0, 3]
    below_c = dist[:, 3, 0]*dist[:, 3, 1]*dist[:, 3, 2]
    between = dist[:, 1, 2]*(dist[:, 3, 0]*dist[:, 3, 1]) + (1-dist[:, 1, 2])*(1-dist[:, 0, 2]*dist[:, 0, 3])
    del dist

    lcs = np.where(normals[:, 1] >= 0, above_b, np.where(normals[:, 2] <= 0, below_c, between))
    return 1-3*np.sum((1-lcs)**2 * factors)/sum(factors), 1-3*np.sum(lcs**2 * factors)/sum(factors)

In [17]:
for i in [10, 50, 100, 200, 300, 400]:
    print(compute_bounds(i))

/tmp/ipykernel_79185/676606884.py:23: RuntimeWarning: divide by zero encountered in divide
  dist = normals[:, :, None]/(normals[:, :, None] - normals[:, None, :]) #distance
/tmp/ipykernel_79185/676606884.py:23: RuntimeWarning: invalid value encountered in divide
  dist = normals[:, :, None]/(normals[:, :, None] - normals[:, None, :]) #distance
/tmp/ipykernel_79185/676606884.py:26: RuntimeWarning: invalid value encountered in multiply
  between = dist[:, 1, 2]*(dist[:, 3, 0]*dist[:, 3, 1]) + (1-dist[:, 1, 2])*(1-dist[:, 0, 2]*dist[:, 0, 3])
/tmp/ipykernel_79185/676606884.py:26: RuntimeWarning: invalid value encountered in add
  between = dist[:, 1, 2]*(dist[:, 3, 0]*dist[:, 3, 1]) + (1-dist[:, 1, 2])*(1-dist[:, 0, 2]*dist[:, 0, 3])


(-0.4932511963007895, 0.38643237750464454)
(-0.09455442011441062, 0.08609079883193005)
(-0.046305924544508326, 0.04410227649635523)
(-0.022415468174775466, 0.02280162882479464)
(-0.014488926728409401, 0.015657870636803684)
(-0.010532772540749136, 0.012077985153147952)


In [4]:
def f(x, y, z, ssb_matrix):
    normal = x*ssb_matrix[0, :] + y*ssb_matrix[1, :] + z*ssb_matrix[2, :] + (1-x-y-z)*ssb_matrix[3, :]
    if normal[1] >= 0:
        ab = normal[0]/(normal[0] - normal[1])
        ac = normal[0]/(normal[0] - normal[2])
        ad = normal[0]/(normal[0] - normal[3])
        return (1 - ab*ac*ad)**2
    elif normal[2] <= 0:
        da = normal[3]/(normal[3] - normal[0])
        db = normal[3]/(normal[3] - normal[1])
        dc = normal[3]/(normal[3] - normal[2])
        return (da*db*dc)**2
    else:
        da = normal[3]/(normal[3] - normal[0])
        db = normal[3]/(normal[3] - normal[1])
        ac = normal[0]/(normal[0] - normal[2])
        bc = normal[1]/(normal[1] - normal[2])
        return (bc*da*db + (1-bc)*(1-ac*(1-da)))**2

In [62]:
# Larger error than tplquad below; therefore not used.
def bounds_z(x, y, _):
    return[0, 1-x-y]
def bounds_y(x, _):
    return[0, 1-x]
def bounds_x(_):
    return[0, 1]

integrate.nquad(f, [bounds_z, bounds_y, bounds_x], args = [ssb_utility], opts={'epsabs': 1e-5, 'epsrel': 1e-5, 'limit': 100}, full_output=True)

(0.05548294891287989, 9.74570978281313e-06, {'neval': 16443})

In [ ]:
val, err = integrate.tplquad(f, 0, 1, 0, lambda x: 1-x, 0, lambda x, y: 1-x-y, args=(ssb_utility,))
1-18*val, 18*err

(0.0013067306856884287, 2.6676784653160325e-07)

In [23]:
def f_general(x, y, z, ssb_matrix):
    normal = x*ssb_matrix[0, :] + y*ssb_matrix[1, :] + z*ssb_matrix[2, :] + (1-x-y-z)*ssb_matrix[3, :]
    vertices_lcs = list(np.where(normal > 0)[0])
    vertices_ucs = [i for i in range(4) if i not in vertices_lcs]
    if len(vertices_lcs) == 0:
        return 0
    elif len(vertices_lcs) == 1:
        volume = 1
        v_lcs = vertices_lcs[0]
        for i in vertices_ucs:
                volume *= normal[v_lcs]/(normal[v_lcs] - normal[i])
        return volume**2
    elif len(vertices_lcs) == 2:
        dist = normal[vertices_lcs, None]/(normal[vertices_lcs, None] - normal[None, vertices_ucs])
        # (bc*da*db + (1-bc)*(1-ac*(1-da)))**2
        return ((1-dist[0, 0])*dist[1, 0]*dist[1, 1] + dist[0, 0]*(1-(1-dist[0, 1])*(1-dist[1, 1])))**2
    elif len(vertices_lcs) == 3:
        volume = 1
        v_ucs = vertices_ucs[0]
        for i in vertices_lcs:
                volume *= normal[v_ucs]/(normal[v_ucs] - normal[i])
        return (1-volume)**2
    elif len(vertices_lcs) == 4:
        return 1

In [22]:
normal = np.array([0.5, -0.8, -0.1, 0.4])
vertices_lcs = list(np.where(normal > 0)[0])
vertices_ucs = [i for i in range(4) if i not in vertices_lcs]
dist = normal[vertices_lcs, None]/(normal[vertices_lcs, None] - normal[None, vertices_ucs])
dist

array([[0.38461538, 0.83333333],
       [0.33333333, 0.8       ]])

In [28]:
f(0, 0.6, 0.4, ssb_utility), f_general(0, 0.6, 0.4, ssb_utility)

(0.308421556122449, 0.30842155612244887)

In [30]:
val, err = integrate.tplquad(f_general, 0, 1, 0, lambda x: 1-x, 0, lambda x, y: 1-x-y, args=(ssb_utility,))
val, 1-18*val, 18*err

(0.055482959406350646, 0.0013067306856884287, 2.6676784653160325e-07)

In [32]:
# 4 cycle
ssb_intr1 = np.array([[0, 1, 1, -1], [-1, 0, 1, 1], [-1, -1, 0, 1], [1, -1, -1, 0]])
# 3 cycle preferred to the last option
ssb_intr2 = np.array([[0, 1, -1, 1], [-1, 0, 1, 1], [1, -1, 0, 1], [-1, -1, -1, 0]])
# 3 cycle not preferred to the last option
ssb_intr3 = np.array([[0, 1, 1, 1], [-1, 0, 1, -1], [-1, -1, 0, 1], [-1, 1, -1, 0]])

In [33]:
val, err = integrate.tplquad(f_general, 0, 1, 0, lambda x: 1-x, 0, lambda x, y: 1-x-y, args=(ssb_intr1,))
val, 1-18*val, 18*err

(0.04881321518812862, 0.12136212661368484, 2.6819671319893127e-07)

In [34]:
val, err = integrate.tplquad(f_general, 0, 1, 0, lambda x: 1-x, 0, lambda x, y: 1-x-y, args=(ssb_intr2,))
val, 1-18*val, 18*err

(0.052889348589358155, 0.047991725391553164, 2.679700612686233e-07)

In [35]:
val, err = integrate.tplquad(f_general, 0, 1, 0, lambda x: 1-x, 0, lambda x, y: 1-x-y, args=(ssb_intr3,))
val, 1-18*val, 18*err

(0.052889348379349846, 0.0479917291717028, 2.6798051586401983e-07)